In [1]:
import os
import json
import shutil
import pickle
import random
import scipy.signal as signal
import numpy as np
from pydub import AudioSegment
import matplotlib.pyplot as plt
from moviepy import VideoFileClip, AudioFileClip, CompositeAudioClip
from pydub.generators import Triangle
from utils.video_demo import make_click_video

def load_pickle(filepath):
    with open(filepath, "rb") as f:
        json_data = pickle.load(f)
    return json_data

with open("genre_symbols_mapping.json", "r") as file:
    genre_id_to_name = json.load(file)

In [2]:
a,b = 45,140
fps = 60
root_dir = "./tempo_estimation_output"
anchor_dir = os.path.join(root_dir, f"tempo_{a}_{b}", "eval_data", "multi")
file_path = os.path.join(anchor_dir, "anchor_zero_uni.pkl")

data = load_pickle(file_path)
ky = 'bothhand_y_bothfoot_y_torso_y_uni'

hit_idx = data[ky]["hit_idx"]
hit_bpm = data[ky]["hit_bpm"]
hit_beat_pulse = data[ky]["hit_beat_pulse"]
hit_ref_bpm = data[ky]["hit_ref_bpm"]
hit_seg_names = data[ky]["hit_seg_names"]
hit_anc_seq = data[ky]["hit_anc_seq"]      
hit_genres = data[ky]["hit_genres"]
filename = data[ky]["filename"]



In [ ]:
for v_idx in range(len(hit_idx)):
    make_click_video(v_idx, data, fps=60)

MoviePy - Building video ./aist_dataset/video_examples/with_clicks/0_Krump_both_foot_y_video_click.mp4.
MoviePy - Writing audio in 0_Krump_both_foot_y_video_clickTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video ./aist_dataset/video_examples/with_clicks/0_Krump_both_foot_y_video_click.mp4



MoviePy - Done !
MoviePy - video ready ./aist_dataset/video_examples/with_clicks/0_Krump_both_foot_y_video_click.mp4
MoviePy - Building video ./aist_dataset/video_examples/with_audio/0_Krump_both_foot_y_video_audio_click.mp4.
MoviePy - Writing audio in 0_Krump_both_foot_y_video_audio_clickTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video ./aist_dataset/video_examples/with_audio/0_Krump_both_foot_y_video_audio_click.mp4



MoviePy - Done !
MoviePy - video ready ./aist_dataset/video_examples/with_audio/0_Krump_both_foot_y_video_audio_click.mp4
✅ Exported click videos for gKR_sBM_c01_d28_mKR3_ch03 (Krump, both_foot_y)
MoviePy - Building video ./aist_dataset/video_examples/with_clicks/1_House_torso_y_video_click.mp4.
MoviePy - Writing audio in 1_House_torso_y_video_clickTEMP_MPY_wvf_snd.mp4


...

MoviePy - Done !
MoviePy - video ready ./aist_dataset/video_examples/with_audio/271_LA style Hip-hop_torso_y_video_audio_click.mp4
✅ Exported click videos for gLH_sBM_c01_d17_mLH4_ch07 (LA style Hip-hop, torso_y)
MoviePy - Building video ./aist_dataset/video_examples/with_clicks/272_Break_both_hand_y_video_click.mp4.
MoviePy - Writing audio in 272_Break_both_hand_y_video_clickTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video ./aist_dataset/video_examples/with_clicks/272_Break_both_hand_y_video_click.mp4



MoviePy - Done !
MoviePy - video ready ./aist_dataset/video_examples/with_clicks/272_Break_both_hand_y_video_click.mp4
MoviePy - Building video ./aist_dataset/video_examples/with_audio/272_Break_both_hand_y_video_audio_click.mp4.
MoviePy - Writing audio in 272_Break_both_hand_y_video_audio_clickTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video ./aist_dataset/video_examples/with_audio/272_Break_both_hand_y_video_audio_click.mp4



frame_index:  59%|█████▉    | 425/719 [00:04<00:03, 91.96it/s, now=None] 

In [ ]:
v_idx = random.randint(1, len(hit_idx)-1)
# fname = filename[v_idx].replace("cAll", "c01")
# anchor_sequence = hit_anc_seq[v_idx].flatten()
# gen = genre_id_to_name.get(str(hit_genres[v_idx]), "Unknown")
# hit_seg = hit_seg_names[v_idx]
# beat_pulse = hit_beat_pulse[v_idx]

# # load video with audio
# video_dir = "./aist_dataset/video"
# video_path = os.path.join(video_dir, fname + ".mp4")
# video = VideoFileClip(video_path)
# duration = video.duration           # in seconds

# print("v_idx:", v_idx)
# print(f"Estimated bpm: {hit_bpm[v_idx]}")
# print(f"Reference bpm: {hit_ref_bpm[v_idx]}")
# print(f"Genre:", gen)
# print(f"Video duration: {duration} seconds")


# subplot to compare original and aligned
# plt.figure(figsize=(15, 6))
# plt.subplot(2, 1, 1)
# plt.title(f"Original Beat Pulse vs Anchor Sequence - {hit_bpm[v_idx]} BPM | {hit_seg} | Genre: {gen}")
# plt.plot(beat_pulse[:len(anchor_sequence)], label="Original Beat Pulse", alpha=0.7)
# plt.plot(anchor_sequence, label="Anchor Sequence", alpha=0.7)
# plt.show()

In [ ]:
make_click_video(v_idx, data, fps=60)


In [ ]:
time = np.arange(len(anchor_sequence)) / 60
peaks, _ = signal.find_peaks(beat_pulse[:len(anchor_sequence)], height=0.1, distance=fps/6)
onset_times = time[peaks] 


click_duration = 25  # milliseconds
click_freq = 1200  # Hz

# Generate a single click sound
click = Triangle(click_freq).to_audio_segment(duration=click_duration)
total_duration = (len(anchor_sequence)/60)*1000  #  in milliseconds

audio_click = AudioSegment.silent(duration=total_duration)
for onset in onset_times:
    position = int(onset * 1000)  # Convert onset time to milliseconds
    audio_click = audio_click.overlay(click, position=position)

click_path = os.path.join("./aist_dataset/video_examples/wavs", f"{v_idx}_clicktrack.wav")
audio_click.export(click_path, format="wav")


click_audio = AudioFileClip(click_path)
# Merge the click audio with the video's original audio
merged_no_audio = video.without_audio().with_audio(click_audio)
merged_no_audio.write_videofile(
    os.path.join("./aist_dataset/video_examples/with_clicks", f"{v_idx}_{gen}_{hit_seg}_video_click.mp4"),
    codec="libx264",
    audio_codec="aac",
    fps=video.fps
)

# Merge the click audio with the video's original audio
combined_audio = CompositeAudioClip([video.audio, click_audio])
merged_with_audio = video.with_audio(combined_audio)
merged_with_audio.write_videofile(
    os.path.join("./aist_dataset/video_examples/with_audio", f"{v_idx}_{gen}_{hit_seg}_video_audio_click.mp4"),
    codec="libx264",
    audio_codec="aac",
    fps=video.fps
)

In [ ]:
# Delete directory contents only
# go inside directory
# rm -rf *


## Genre Clustering

In [ ]:
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from utils.dance_tempo_pipeline import (get_all_anchor_paths,
                                        parse_filename,
                                        load_marker_data,
                                        compute_combinations,
                                        load_resultant)


fps = 60
w_sec = 5
h_sec = w_sec/2
window_size = int(fps*w_sec)
hop_size = int(fps*h_sec)

a = 45 
b = 140
output_dir = "tempo_estimation_output"

def get_ioi(seq):
        onsets = np.where(seq == 1)[0] / fps
        return np.diff(onsets)
    
with open("genre_symbols_mapping.json", "r") as file:
    genre_id_to_name = json.load(file)
    
save_dir = os.path.join(output_dir, f"tempo_{a}_{b}/")


aist2d_path = "./aist_dataset/aist_annotation/keypoints2d"
aist_filelist = os.listdir(aist2d_path)

genre_iois = {}
record_iois = []
for filename in tqdm(aist_filelist):
    paths = get_all_anchor_paths("anchor_zero", "uni", filename)
    if not os.path.exists(paths["markers"]["left_wrist"]["ax0"]):
        continue

    meta = parse_filename(filename)
    data = load_marker_data(paths)
    combined = compute_combinations(data, fps, thres=0.2)
    
    torso_seq = data['torso_y']     # extracted rhythmic anchors, binary sequences
    hands_seq = combined['both_hand_y'] # extracted rhythmic anchors, binary sequences
    feet_seq = combined['both_foot_y'] # extracted rhythmic anchors, binary sequences
    genre = genre_id_to_name[meta["dance_genre"]]


    
    